### From point clouds to graphs

## Miguel Vaz

### Düsseldorf, 28th October 2015

# About me

Finance pays my bills

d-fine
360 T

PhD in Robotics (speech interaction with a humanoid) brought me there
Honda Research Institute Europe

@migueljvaz

I don't like "Alt"

<div class="col-xs-1"><h1>I</h1></div>
<div class="col-xs-2">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/Heart_coraz%C3%B3n.svg/75px-Heart_coraz%C3%B3n.svg.png"/>
</div>
<div class="col-xs-6"><h1>networks</h1></div>

say something

# Why?

Networks give us a way to quantify and reason about concepts that are already familiar to us.

"Frankfurt airport is an international hub"

"Lehman was too central to fail"

"Thomas Wiecki is a well connected data scientist"

![](http://ecx.images-amazon.com/images/I/71qAmSP21wL.jpg)

Mention paper from PyData 2014

In [ ]:
# preparation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import  display, display_html

# Part 1: point clouds

Let's assume a well behaved dataset, with points selected from either of the following distributions

$X_1 \sim N\left( \mu=(1,1),\, \sigma^2=(0.9, 0.9) \right)$

$X_2 \sim N\left( \mu=(-0.5,0),\, \sigma^2=(0.9, 0.9) \right)$

$X_3 \sim N\left( \mu=(1,-1),\, \sigma^2=(0.9, 0.9) \right)$

Well-behaved means in this case that it is described by almost non-overlaping centroids.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import output_notebook, figure, show, ColumnDataSource
TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,tap,previewsave,box_select,poly_select,lasso_select"
output_notebook()

In [ ]:
from kmeans_aux import *

In [ ]:
from sympy import *
init_printing(use_unicode=True)

In [ ]:
m = Matrix(np.matrix([[-2,1,1], [2,-2,0], [1,0,-1]]) )
m

In [ ]:
X, labels_true, centres = generate_three_circles()
src = create_datasource(X, labels_true, centres)

In [ ]:
p = figure(tools=TOOLS);
p.scatter('x', 'y', radius='radius', color='fill_color', source=src);
show(p)

## K-Means

One possible way of recovering the categories is by using the K-Means clustering algorithm.

The $k$-means uses $k$ centroids to define clusters.
It finds the best centroids by alternating between

1. assigning data points to clusters based on the current centroids
2. chosing centroids (points which are the center of a cluster) based on the current assignment of data points to clusters.

![xxx](pics/kmeans.png)

Picture taken from [this website](http://stanford.edu/~cpiech/cs221/handouts/kmeans.html) by Chris Piech.

1. Initialize cluster centroids $\mu_1, \mu_2, \ldots, \mu_k \in \mathcal{R}^n$ (randomly)
2. Repeat until convergence : {

    For every $i$, set $$c^{(i)} := arg \min_{j} || x^(i) - \mu_j ||^2$$

    For each $j$, set $$\mu_j := \frac{\sum_{i=1}^m 1_{c^(i) = j}x^(i) }{\sum_{i=1}^m 1_{c^(i) = j}x^(i)}$$
}

Scikit-learn already has an implementation of the algorithm

In [ ]:
k_means_3_labels, k_means_3_cluster_centres = do_kmeans(X, n_clusters=3)
distance, order, is_correct = compare_kmeans_with_prior(k_means_3_cluster_centres, k_means_3_labels, centres, labels_true)

In [ ]:
# part the plot
alpha = 0.9  - is_correct * 0.8
rad = src.data['radius'] * (1 + (~ is_correct * 1))
src.add(rad, 'radius2')
src.add(alpha, name='alpha' )

In [ ]:
p = figure(tools=TOOLS);
p.scatter('x', 'y', color='fill_color', alpha='alpha', radius='radius2', source=src);
show(p)

# Part 2: graphs

Let's assume a different dataset, with points selected from either of the following distributions.

In [ ]:
X, true_labels = generate_two_moons()
p = figure(tools=TOOLS);
x,y = [list(t) for t in zip(*X)]
p.scatter(x, y, color='black', radius=0.02);
show(p)

The distribution is no longer described by centroids.

K-Means does not capture the non-linearity of the dataset structure

In [ ]:
k_means_2_labels, k_means_2_cluster_centres = do_kmeans(X, n_clusters=2, n_init=10)
src2 = create_datasource(X, k_means_2_labels, k_means_2_cluster_centres)

In [ ]:
p = figure(tools=TOOLS);
p.scatter('x', 'y', color='fill_color', source=src2);
# alpha='alpha', radius='radius', 
show(p)

# Other solutions?

### Neural Network

### Project the dataset into a different space (Support Vector Machine)

### Any suggestions?




### Today we will look at Spectral Clustering

In [ ]:
from d3networkx import EventfulGraph, ForceDirectedGraph, empty_eventfulgraph_hook

In [ ]:
from networkx.generators import random_graphs
from networkx.generators import classic

In [ ]:
def handle_graph(graph):
    print(graph.graph._sleep)
    graph_widget = ForceDirectedGraph(graph, width=600, height=500)
    display(graph_widget)
EventfulGraph.on_constructed(handle_graph)
random_graphs.empty_graph = empty_eventfulgraph_hook(sleep=0.01)

In [ ]:
g = random_graphs.powerlaw_cluster_graph(70, 3, 0.7);

In [ ]:
g.node[1]['fill'] = '#770000'